In [1]:
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))
print(text[:100])

corpus length: 600893
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all ph


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 200285


In [5]:
# One hot encoding of input and output
x = np.zeros((len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print(y[:4])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index of most probable char from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    # random start index to select the starting input
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        # Extracting text from file using random start index and upto length of maxlen
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            # One hot encoding of input text 
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # Predicting the new char
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
    
            # Updating the sentence to generate new char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [9]:
# Callback function to print the generayed output after each epoch
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
# Callback to save the model with least loss
filepath="text_generation2_model-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [10]:
# Fit the model. Gernerate the file to save best model and also generate the text using calbacks after eack epoch
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback, checkpoint])

Epoch 1/60
200285/200285 [==============================] - 95s 475us/step - loss: 1.9632

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "hat is manifestly believed with due impl"
hat is manifestly believed with due implease the sense of the sense of the sense of the something to the stranger of the something to the something to the most of the sense of the something to the sense of the so the sense of the something to the something to the stronger to the something of the senses of the sense of the strinting and the senses and or the something to the sense of the sense of the something to the sense of the strong 
----- diversity: 0.5
----- Generating with seed: "hat is manifestly believed with due impl"
hat is manifestly believed with due impleated in the
discive and itself, in it is is this "so a more and gains of the most of the desire of the spority
enonged of the schored to german more of the profound and and his one to even the more or and t

conjectured, become, and of the of the simplity of certain the same forturiations, and good will and world and should be for the word and of the her at perhaps and morality, a stands the he is not be so pleasures and the conscious the things to his the still it is the stands of the historical entire, supposing, the partinces, for a surpinization to be the truths, how still in his destracimate the commanding in the res
----- diversity: 1.0
----- Generating with seed: "precedent, as thus
conjectured, become, "
precedent, as thus
conjectured, become, and
life! it is, frentness, hy be naive the
like fundamental of its remain," hos his after are life his filling
incantior hadded to amour
longer not but net fundamentarization of the closed, it more a possibly right in the qualities who east destance, the good rascip bito, as smortthering, but the indeed, presse of the straighty of 9manifical rature for shord parpospy to
the first is invent in the
----- diversity: 1.2
----- Generating with se

the question: did a woman herself ever a know rightsy, that the sufferst, thbuilicated in hypotrs exdetent of a
philosopherly are this brong human, as, and plus, be-paltated to now nakeche--.wish esward, nothive what and stailes--neithel ptilogled to barnizator inaggertnesser--almost revence in theirs.--, underable , itse speep
outhiniry barns tonourad
"deterious"puda not "ioner", that betting words; toknes, and nowadgement are bind hei

Epoch 00007: loss improved from 1.43318 to 1.41485, saving model to text_generation2_model-07-1.4149-bigger.hdf5
Epoch 8/60
200285/200285 [==============================] - 81s 406us/step - loss: 1.3999

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "in contrast to all sorts of mystics,
who"
in contrast to all sorts of mystics,
who has been the love of the probably the stands of the exceptions of the stands of a still in the strange of the long and hence a stands and the strange of the strictic of the stricted, a

 aristocratic morality is intolerant in the senses of the subjecting the super-the soul of the struggle of the problem of the subjecting and and senses and the senses of the soul of the soul of the world," and and and all the subjecting and the subjecting the subjecting the spirit of the sensations and the soul of the subjecting the subjecting and the world and the sublimes of the problem of the senses of the problem of the subjecting t
----- diversity: 0.5
----- Generating with seed: " aristocratic morality is intolerant in "
 aristocratic morality is intolerant in the anti-hitherbornation, and is the most soul of the probable of the strong of the sublimest and and and the understanding of the subjection and soul
same time responsibilarianing the sometimation of the principle could that the freedom of the strangly such a philosophy of the most denies of his conditional taste, and the problem of the fact that is a conditional truths and more and humanity of t
----- diversity: 1.0
-----

en and unto himself, as in the natural succeving circumstanct genmines and we think and who more enterticistad woll arong tor taste wh get
our to intented in impeents and concers
away
beinged that is their decision of the
apting it? us evolation of preout, syllingms as morals for
nothing plato, everything ears and a girs have been
this more intrantiture when the occurea self-would love away
that no longer qualities: he, it
is learned to
----- diversity: 1.2
----- Generating with seed: "en and unto himself, as in the natural s"
en and unto himself, as in the natural saev, which usfeptically knows yousthing rquity ambabolst, and renerst itselfiness constantly that can be courage be don
unlely not wovening herey estimism.
ever fis who moking a cases are at ecolecoling for can
tern avoir in the porition
of strange have
setinal voly a tempt of happistsing othary for evolues.
piltly logicism
of
the ewormotoles
of wimats whether is, becemp we spore
takecties, and un

Epoch 00014: loss improve

...

Epoch 44/60
200285/200285 [==============================] - 115s 574us/step - loss: 1.2577

----- Generating text after Epoch: 43
----- diversity: 0.2
----- Generating with seed: "owever, because he is unselfish, but bec"
owever, because he is unselfish, but because the conscience of the same thing and in the most suffering and sense-and the sensible, and the power and proposition of the sense, and the strength of the powerful and the words of the sense, and the sense of the power of the standards and sufficient and provident the propered and a state of the subjeching and the sense, and the religion of all the conscience of the sense, and the powerful ha
----- diversity: 0.5
----- Generating with seed: "owever, because he is unselfish, but bec"
owever, because he is unselfish, but because he is also the words and deception and possibility of religion to form to be did not believed as a find supposing a distrusts seems to use in the terribly of the demonstranis and everything which we 

the portender in order to account the greater conception with their nothing and prospection will perhaps not be more process
----- diversity: 1.0
----- Generating with seed: "her aside. for it is, to all appearances"
her aside. for it is, to all appearances
of victless, and let us his particle been christian to ssys strengtad we him, birstly conflict distasted ervices confinish and
honour it,
stands instinctive,
to taporning order to relight the brol in
stepine. that of in the intertous as will some tywanding the virtus
conduct, or whate upon
happiness, and experience is actually permimlialilis-porilence by all side-certain happery, my, within the a
----- diversity: 1.2
----- Generating with seed: "her aside. for it is, to all appearances"
her aside. for it is, to all appearances moralitynace,
or herch' go of the century
awece, or acciphed and ladder
from callst
enough ordes. heren of thinkfrigal;
ojest of sensation of
moral, bacony, glances, as i fingered, just revedutly. capaminess, 

mohtuest, ,tecy. in the world: and wishes lasting eor hoyolea: however, science! on the "rancibilies." i mojest or gradeument, and among every
comfoun inned philosophers advahtac age that

Epoch 00050: loss improved from 1.24923 to 1.24915, saving model to text_generation2_model-50-1.2491-bigger.hdf5
Epoch 51/60
200285/200285 [==============================] - 85s 426us/step - loss: 1.2475

----- Generating text after Epoch: 50
----- diversity: 0.2
----- Generating with seed: "t here is where the delusion
prevails; i"
t here is where the delusion
prevails; it is the consciously and something himself and and the same time of the strange of the same time and the same time and and and and of the desire and and delight of the power and should be the soul of the strange, and all the same as the speaks and interpretation of the stare of the soul and self-interest and proposition of the sense, and all the consciously and and as the same as a philosophy and 
----- diversity: 0.5
----- Generati

is quickly approaching, that everything which is a certain of determines of the power, or that the
worst and possible as the commanding and prevalument and the lightness of the valuations of the spirituality and standard of the most
such a truth that the best from the personalists of the such soul of the "good and thought," at all the best contemplation of the presently the condition of men of the strange, as the contemplation that a pe
----- diversity: 1.0
----- Generating with seed: "is quickly approaching, that everything "
is quickly approaching, that everything other
ciddge to egrous venies me simply even his danger at
least to
list may suppose has, to make
life the instrumente? for these "acceptes"; is the prevail yet indeceive doisen as man will to the question--is
extravagant!

othechs that it
genius moll! green order of companned must
be that person that the
year great which is respectly also a payt use mis!


6utacide philosophy that there is danger
----- diversity: 1.2
-----

dear,
inedicatable in a alressnos or to
last, furrore. good
virluen-things at the soul and conhicist, erron just to their other
individual yet what; "ill, even as godful each "hoverire colling," feugifically
toly
wanting ayronous, except a man, and
brough, who, my watwiso
is but he
occassoritable., is not
trmume gage, with his decising these the threand, threid justice secret applient, proud
than 

Epoch 00057: loss improved from 1.24010 to 1.23909, saving model to text_generation2_model-57-1.2391-bigger.hdf5
Epoch 58/60
200285/200285 [==============================] - 90s 452us/step - loss: 1.2395

----- Generating text after Epoch: 57
----- diversity: 0.2
----- Generating with seed: "ispensable for the morrow and the day af"
ispensable for the morrow and the day after the sense of the same time and the morality of the senses of the sense of the instinct of the sense, and it is not be a suffering and the strange of the experience of the senses of the prostent and process of the sense 